In [1]:
from IPython.core.events import EventManager
from IPython import get_ipython
import time

global_start_time = None

def start_timer(event):
    global global_start_time
    global_start_time = time.time()

def stop_timer(event):
    global global_start_time
    if global_start_time:
        duration = time.time() - global_start_time
        print(f"Cell execution time: {duration:.3f} seconds")
        global_start_time = None

ip = get_ipython()
ip.events.register('pre_run_cell', start_timer)
ip.events.register('post_run_cell', stop_timer)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from skimage import io 
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mplPolygon
from glob import glob
import cv2
from scipy import ndimage
import matplotlib.colors as mcolors
from scipy.spatial import KDTree
from sklearn.neighbors import NearestNeighbors
from scipy import ndimage as ndi

Cell execution time: 1.883 seconds


In [3]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="1 PTB-22.1"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.005 seconds


In [4]:
mat = io.imread("../img/1 PTB-22.1.tiff")

Cell execution time: 0.261 seconds


In [5]:
x_221_sf = (11008-10172)/(42949.5-39599)
y_221_sf = (3569-1523)/(19294.5-11100)
x_221_cnst = 10172-39599*x_221_sf
y_221_cnst = 1523-11100*y_221_sf

Cell execution time: 0.000 seconds


In [6]:
comp = glob("../composite/PTB221*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [7]:
xw =128

for spot in df.iterrows():
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=np.round((x*x_221_sf)+x_221_cnst).astype(int)
    y=np.round((y*y_221_sf)+y_221_cnst).astype(int)
    img_bb = mat[(x-xw):(x+xw),(y-xw):(y+xw)]
    white_pixels = np.where(np.all(img_bb == [255, 255, 255, 255], axis=-1))

    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([128,128])).sum(1))

    selected_pixels = np.array(white_pixels).T

    nearest_neighbors = NearestNeighbors(n_neighbors=5)
    nearest_neighbors.fit(selected_pixels)
    neighbors = nearest_neighbors.kneighbors(selected_pixels, return_distance=False)
    neighbors = [set(list(x)) for x in neighbors]
    
    pp = neighbors[starting_point]
    add_point = True
    while (add_point):
        old_pp = pp
        pp = set.union(*[neighbors[x] for x in pp])
        if old_pp == pp:
            add_point=False
    pp=list(pp)
    white_pixels = tuple([x[pp] for x in white_pixels])
    
    img_comp = io.imread(comp[roi])

    x_coord,y_coord=white_pixels

    img_sf = img_bb.shape[0]/img_comp.shape[0]
    img_sf = img_sf

    x_coord=x_coord/img_sf
    y_coord=y_coord/img_sf

    x_coord = x_coord.astype(int)
    y_coord = y_coord.astype(int)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_comp,alpha=0)
    plt.scatter(y_coord,x_coord,s=100,c="black",alpha=1)
    plt.axis("off")
    fig.patch.set_visible(False)
    plt.savefig('scatter.png', bbox_inches='tight', pad_inches=0)  # Save without padding
    plt.close(fig)  # Close plot

    image = cv2.imread('scatter.png', cv2.IMREAD_GRAYSCALE)  # Load as grayscale image for simplicity

    smooth_image = ndimage.gaussian_filter(image, sigma=10)
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED)  # Load with alpha channel

    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)]  # RGB and alpha values for each point in the colormap
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=5)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=5)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=25)
    
    np.save("../bb_mask/"+"PTB221_"+(str(roi))+".npy",final_mask)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_bb)
    plt.axis("off")
    plt.savefig("../bb_annot/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]

    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB221_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 294.029 seconds


In [8]:
df = pd.read_excel("../data/geomx_221_223.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="2 PTB-22.3"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.752 seconds


In [9]:
mat = io.imread("../img/2 PTB-22.3-2.tiff")

Cell execution time: 0.257 seconds


In [10]:
x_223_sf = (4318-6234)/(16205.8-23872)
y_223_sf = (3344-5892)/(18429.5-28631)
x_223_cnst = 4318-16205.8*x_221_sf
y_223_cnst = 3344-18429.5*y_221_sf

Cell execution time: 0.000 seconds


In [11]:
comp = glob("../composite/PTB223*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [12]:
for spot in df.iterrows():
    # get spot attributes 
    y,x=spot[1][["ROICoordinateX","ROICoordinateY"]]
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    x=np.round((x*x_223_sf)+x_223_cnst).astype(int)
    y=np.round((y*y_223_sf)+y_223_cnst).astype(int)
    xw =128
    
    img_bb = mat[(x-xw):(x+xw),(y-xw):(y+xw)]
    white_pixels = np.where(np.all(img_bb == [255, 255, 255, 255], axis=-1))

    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([128,128])).sum(1))

    selected_pixels = np.array(white_pixels).T

    nearest_neighbors = NearestNeighbors(n_neighbors=5)
    nearest_neighbors.fit(selected_pixels)
    neighbors = nearest_neighbors.kneighbors(selected_pixels, return_distance=False)
    neighbors = [set(list(x)) for x in neighbors]
    
    pp = neighbors[starting_point]
    add_point = True
    while (add_point):
        old_pp = pp
        pp = set.union(*[neighbors[x] for x in pp])
        if old_pp == pp:
            add_point=False
    pp=list(pp)
    white_pixels = tuple([x[pp] for x in white_pixels])
    
    img_comp = io.imread(comp[roi])

    x_coord,y_coord=white_pixels

    img_sf = img_bb.shape[0]/img_comp.shape[0]
    img_sf = img_sf

    x_coord=x_coord/img_sf
    y_coord=y_coord/img_sf

    x_coord = x_coord.astype(int)
    y_coord = y_coord.astype(int)
     
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_comp,alpha=0)
    plt.scatter(y_coord,x_coord,s=100,c="black",alpha=1)
    plt.axis("off")
    fig.patch.set_visible(False)
    plt.savefig('scatter.png', bbox_inches='tight', pad_inches=0)  # Save without padding
    plt.close(fig)  # Close plot

    image = cv2.imread('scatter.png', cv2.IMREAD_GRAYSCALE)  # Load as grayscale image for simplicity

    smooth_image = ndimage.gaussian_filter(image, sigma=10)
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED)  # Load with alpha channel

    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)]  # RGB and alpha values for each point in the colormap
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=5)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=5)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=25)
    
    np.save("../bb_mask/"+"PTB223_"+(str(roi))+".npy",final_mask)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_bb)
    plt.axis("off")
    plt.savefig("../bb_annot/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB223_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 168.701 seconds


In [13]:
# PTB 22.2 already has bounding boxes around each spot 
df = pd.read_excel("../data/geomx_222.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="PTB22.2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 1.279 seconds


/Users/jperrie/miniconda3/envs/scvi/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [14]:
comp = glob("../composite/PTB222*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [15]:
for spot in df.iterrows():
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    img_bb = io.imread("../bb_annot/PTB222_"+str(roi)+".png")
    img_bb = img_bb[:img_bb.shape[1],:img_bb.shape[1]]
    pad = 0
    img_bb = img_bb[pad:img_bb.shape[1]-pad,pad:img_bb.shape[0]-pad]
    white_pixels = np.where(np.all(img_bb == [255, 255, 255, 255], axis=-1))

    # find starting point closest to center of image 
    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))

    # find all white pixels that are connected to starting point 
    selected_pixels = np.array(white_pixels).T

    nearest_neighbors = NearestNeighbors(n_neighbors=5)
    nearest_neighbors.fit(selected_pixels)
    neighbors = nearest_neighbors.kneighbors(selected_pixels, return_distance=False)
    neighbors = [set(list(x)) for x in neighbors]
    
    pp = neighbors[starting_point]
    add_point = True
    while (add_point):
        old_pp = pp
        pp = set.union(*[neighbors[x] for x in pp])
        if old_pp == pp:
            add_point=False
    pp=list(pp)
    white_pixels = tuple([x[pp] for x in white_pixels])
    
    # load in high quality image 
    img_comp = io.imread(comp[roi])

    x_coord,y_coord=white_pixels

    img_sf = img_bb.shape[0]/img_comp.shape[0]
    img_sf = img_sf

    x_coord=x_coord/img_sf
    y_coord=y_coord/img_sf

    x_coord = x_coord.astype(int)
    y_coord = y_coord.astype(int)
     
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_comp,alpha=0)
    plt.scatter(y_coord,x_coord,s=100,c="black",alpha=1)
    plt.axis("off")
    fig.patch.set_visible(False)
    plt.savefig('scatter.png', bbox_inches='tight', pad_inches=0)  # Save without padding
    plt.close(fig)  # Close plot

    image = cv2.imread('scatter.png', cv2.IMREAD_GRAYSCALE)  # Load as grayscale image for simplicity

    smooth_image = ndimage.gaussian_filter(image, sigma=10)
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED)  # Load with alpha channel

    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)]  # RGB and alpha values for each point in the colormap
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=5)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=5)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=25)

    np.save("../bb_mask/"+"PTB222_"+(str(roi))+".npy",final_mask)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB222_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB222_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 269.880 seconds


In [16]:
df = pd.read_excel("../data/geomx_211.xlsx",sheet_name="SegmentProperties")
df = df.loc[df["SlideName"]=="B172914-2"]
df = df[["ROILabel","ROICoordinateX","ROICoordinateY","AOISurfaceArea"]]

Cell execution time: 0.355 seconds


In [17]:
comp = glob("../composite/PTB211*")
comp = {int(x.split("_")[-1].split(".png")[0]):x for x in comp}

Cell execution time: 0.001 seconds


In [18]:
for spot in df.iterrows():
    roi=spot[1][["ROILabel"]].astype(int).values[0]
    img_bb = io.imread("../bb_annot/PTB211_"+str(roi)+".png")
    img_bb = img_bb[:img_bb.shape[1],:img_bb.shape[1]]
    pad = 0
    img_bb = img_bb[pad:img_bb.shape[1]-pad,pad:img_bb.shape[0]-pad]

    condition1 = np.all(img_bb == [255, 255, 255, 255], axis=-1)
    condition2 = np.all(img_bb == [254, 254, 254, 255], axis=-1)
    condition3 = np.all(img_bb == [254, 255, 255, 255], axis=-1)
    condition4 = np.all(img_bb == [255, 254, 255, 255], axis=-1)
    condition5 = np.all(img_bb == [255, 255, 254, 255], axis=-1)

    # Combine the conditions using logical OR
    combined_condition = np.logical_or(np.logical_or(np.logical_or(np.logical_or(condition1, condition2),condition3),condition4),condition5)

    # Get the indices of white pixels under any of these conditions
    white_pixels = np.where(combined_condition)

    # find starting point closest to center of image 
    starting_point = np.argmin(np.abs(np.array(white_pixels).T-np.array([img_bb.shape[1]/2,img_bb.shape[0]/2])).sum(1))

    # find all white pixels that are connected to starting point 
    selected_pixels = np.array(white_pixels).T

    nearest_neighbors = NearestNeighbors(n_neighbors=5)
    nearest_neighbors.fit(selected_pixels)
    neighbors = nearest_neighbors.kneighbors(selected_pixels, return_distance=False)
    neighbors = [set(list(x)) for x in neighbors]
    
    pp = neighbors[starting_point]
    add_point = True
    while (add_point):
        old_pp = pp
        pp = set.union(*[neighbors[x] for x in pp])
        if old_pp == pp:
            add_point=False
    pp=list(pp)
    white_pixels = tuple([x[pp] for x in white_pixels])
    
    # load in high quality image 
    img_comp = io.imread(comp[roi])

    x_coord,y_coord=white_pixels

    img_sf = img_bb.shape[0]/img_comp.shape[0]
    img_sf = img_sf

    x_coord=x_coord/img_sf
    y_coord=y_coord/img_sf

    x_coord = x_coord.astype(int)
    y_coord = y_coord.astype(int)
     
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(img_comp,alpha=0)
    plt.scatter(y_coord,x_coord,s=100,c="black",alpha=1)
    plt.axis("off")
    fig.patch.set_visible(False)
    plt.savefig('scatter.png', bbox_inches='tight', pad_inches=0)  # Save without padding
    plt.close(fig)  # Close plot

    image = cv2.imread('scatter.png', cv2.IMREAD_GRAYSCALE)  # Load as grayscale image for simplicity

    smooth_image = ndimage.gaussian_filter(image, sigma=10)
    target_image = cv2.imread(comp[roi], cv2.IMREAD_UNCHANGED)  # Load with alpha channel

    resized_smooth_image = cv2.resize(smooth_image, (target_image.shape[1], target_image.shape[0]))
    mask = ~(resized_smooth_image>200)

    colors = [(0, 0, 0, 0), (1, 1, 1, 0.25)]  # RGB and alpha values for each point in the colormap
    cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors, N=2)
    
    gaus_mask = ndimage.gaussian_filter(mask, sigma=5)
    dilate_mask = cv2.dilate(gaus_mask.astype(np.uint8), None, iterations=5)
    final_mask = cv2.erode(dilate_mask.astype(np.uint8), None, iterations=15)

    np.save("../bb_mask/"+"PTB211_"+(str(roi))+".npy",final_mask)
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(target_image, cv2.COLOR_BGRA2RGBA))
    plt.imshow(final_mask,cmap=cmap)
    plt.axis("off")
    plt.savefig("../bb_comp/"+"PTB211_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()
    
    filled_mask = ndi.binary_fill_holes(final_mask)
    masked_image = np.zeros_like(target_image,)
    masked_image[filled_mask == 1] = target_image[filled_mask == 1]
    
    fig,ax=plt.subplots(figsize=(16,16))
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGRA2RGBA))
    plt.axis("off")
    plt.savefig("../bb_comp_mask/"+"PTB211_"+(str(roi)),dpi=400,bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    plt.clf()

<Figure size 640x480 with 0 Axes>

Cell execution time: 239.549 seconds
